# Query Snowflake in Python

In [ ]:
# copy and paste in to a new chunk, enter credentials and run to save in environment. Then delete chunk
%env snowflakeuser=<your_snowflake_username>
%env snowflakepass=<your_snowflake_password>

In [3]:
# Packages
import snowflake.connector
import pandas as pd
import os
import numpy as np

c:\Users\crudek\Anaconda3\lib\site-packages\snowflake\connector\options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [4]:
def snowflake_to_pandas(connection_params, query):
    try:
        # Establish a connection to Snowflake
        conn = snowflake.connector.connect(**connection_params)

        # Execute the SQL query and fetch the results into a DataFrame
        df = pd.read_sql_query(query, conn)

        # Close the connection
        conn.close()

        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

# Snowflake connection parameters
connection_params = {
    "user": os.environ['snowflakeuser'],
    "password": os.environ['snowflakepass'],
    "account": "zib52348.us-east-1",
    "role": "ACCOUNTADMIN",
    "warehouse": "REPORTING",
    "database": "ANALYTICS",
    "schema": "FORECASTING",
}

# SQL command 
query = 'SELECT * FROM "ANALYTICS"."FORECASTING"."sales_fcast_combined_v"'

# Call the function to retrieve the data into a Pandas DataFrame
result_df = snowflake_to_pandas(connection_params, query)

if result_df is not None:
    print(result_df.head())  # Display the first few rows of the DataFrame
else:
    print("Failed to retrieve data from Snowflake.")

c:\Users\crudek\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   DEP_ENT       MONTH    NET_SALES  BUDGET_AMOUNT  FORECAST
0  210_155  2016-12-01   6719381.11     14486537.0       NaN
1  210_155  2017-01-01   1694443.80      6155140.0       NaN
2  210_155  2017-02-01   4319283.82      8460802.0       NaN
3  210_155  2017-03-01   3585583.41     13007481.0       NaN
4  210_155  2017-04-01  15336265.41     15871856.0       NaN


###############
### Phase 1 ###
###############

Lets understand the packages and process being used and forecast for one department-entity combination before doing multiple series

Referecen: https://moez-62905.medium.com/time-series-in-python-10502f9fac2a

In [5]:
# head and tail
print(result_df)

      DEP_ENT       MONTH    NET_SALES  BUDGET_AMOUNT  FORECAST
0     210_155  2016-12-01   6719381.11     14486537.0       NaN
1     210_155  2017-01-01   1694443.80      6155140.0       NaN
2     210_155  2017-02-01   4319283.82      8460802.0       NaN
3     210_155  2017-03-01   3585583.41     13007481.0       NaN
4     210_155  2017-04-01  15336265.41     15871856.0       NaN
...       ...         ...          ...            ...       ...
1260  250_155  2021-11-01  13526148.43            NaN       NaN
1261  250_155  2021-12-01   5335129.60            NaN       NaN
1262  250_155  2022-01-01   5831587.19      5939097.0       NaN
1263  250_155  2022-02-01   5428107.02      7616347.0       NaN
1264  250_155  2022-03-01   5226201.46      7789916.0       NaN

[1265 rows x 5 columns]


In [6]:
result_df["MONTH"] = pd.to_datetime(result_df["MONTH"])
print(result_df.head())
print(result_df.dtypes)

   DEP_ENT      MONTH    NET_SALES  BUDGET_AMOUNT  FORECAST
0  210_155 2016-12-01   6719381.11     14486537.0       NaN
1  210_155 2017-01-01   1694443.80      6155140.0       NaN
2  210_155 2017-02-01   4319283.82      8460802.0       NaN
3  210_155 2017-03-01   3585583.41     13007481.0       NaN
4  210_155 2017-04-01  15336265.41     15871856.0       NaN
DEP_ENT                  object
MONTH            datetime64[ns]
NET_SALES               float64
BUDGET_AMOUNT           float64
FORECAST                float64
dtype: object


In [7]:
df_200_155 = result_df[result_df["DEP_ENT"] == '200_155']
df_200_155 = df_200_155[df_200_155['MONTH'] <= pd.Timestamp(2023,6,1)] 
"""
ABOVE: we have data beyond july '23, but we want to compare with 
some models that only had through june. No way to check model accuracy beyond the daya we have
"""
df_200_155 = df_200_155[["MONTH", "NET_SALES"]]
print(df_200_155)
print(df_200_155.dtypes)

         MONTH   NET_SALES
192 2016-01-01  2592017.04
193 2016-02-01  2348092.91
194 2016-03-01  2894584.98
195 2016-04-01  2163636.39
196 2016-05-01  3210147.13
..         ...         ...
910 2023-02-01  1728206.02
911 2023-03-01  2325654.24
912 2023-04-01  1918474.98
913 2023-05-01  2443940.14
914 2023-06-01  1748262.30

[90 rows x 2 columns]
MONTH        datetime64[ns]
NET_SALES           float64
dtype: object


In [8]:
# ML Can't deal with dates directly so we need to extract some basic properties

# extract month and year from dates
df_200_155['Month'] = [i.month for i in df_200_155['MONTH']]
df_200_155['Year'] = [i.year for i in df_200_155['MONTH']]
# create a sequence of numbers
df_200_155['Series'] = np.arange(1,len(df_200_155)+1)
print(df_200_155.head())

         MONTH   NET_SALES  Month  Year  Series
192 2016-01-01  2592017.04      1  2016       1
193 2016-02-01  2348092.91      2  2016       2
194 2016-03-01  2894584.98      3  2016       3
195 2016-04-01  2163636.39      4  2016       4
196 2016-05-01  3210147.13      5  2016       5


In [9]:
# Create training dataset
train = df_200_155[df_200_155['MONTH'] <= pd.Timestamp(2022,4,1)] # ~80%
# drop unnecessary columns and re-arrange
train.drop(['MONTH'], axis=1, inplace=True)
train = train[['Series', 'Year', 'Month', 'NET_SALES']] 


# Create testing dataset
test = df_200_155[df_200_155['MONTH'] > pd.Timestamp(2022,4,1)] # ~20%
# drop unnecessary columns and re-arrange
test.drop(['MONTH'], axis=1, inplace=True)
test = test[['Series', 'Year', 'Month', 'NET_SALES']] 

C:\Users\crudek\AppData\Local\Temp\ipykernel_22768\3638390786.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['MONTH'], axis=1, inplace=True)
C:\Users\crudek\AppData\Local\Temp\ipykernel_22768\3638390786.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(['MONTH'], axis=1, inplace=True)


In [10]:
# good if around 80/20 split train/test
train.shape, test.shape

((76, 4), (14, 4))

In [11]:
# import the regression module
from pycaret.regression import *

In [12]:
import plotly.express as px

In [19]:
# initialize setup
s = setup(data = train, test_data = test, target = 'NET_SALES', 
          fold_strategy = 'timeseries', numeric_features = ['Year', 'Series'], 
          fold = 3, transform_target = True, session_id = 123)

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:12:19
Status,. . . . . . . . . . . . . . . . . .,Preparing Data for Modeling


ImportError: cannot import name '_fit_context' from 'sklearn.base' (c:\Users\crudek\Anaconda3\lib\site-packages\sklearn\base.py)

In [14]:
best = compare_models(sort = 'MAPE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,693941.7065,1023931896165.8785,945953.4551,0.2613,0.3270,0.2359,0.0067
ridge,Ridge Regression,745730.4792,1292525742762.6667,1025399.4792,0.1223,0.3929,0.2379,0.5967
en,Elastic Net,799096.1250,1474958677333.3333,1161530.3958,-0.1143,0.3818,0.2477,0.6000
huber,Huber Regressor,927479.8130,2134064719976.5935,1416306.6735,-0.6676,0.4919,0.2555,0.0067
lr,Linear Regression,779685.3125,1393241467562.6667,1073427.4792,0.0373,0.5108,0.2695,1.2900
lasso,Lasso Regression,783918.3333,1398150059349.3333,1076787.6042,0.0310,0.5175,0.2717,0.5967
llar,Lasso Least Angle Regression,783917.5625,1398143767893.3333,1076786.8542,0.0310,0.5175,0.2717,0.5567
par,Passive Aggressive Regressor,919311.1697,1886329311696.8767,1305202.0655,-0.4072,0.4568,0.2906,0.0033
br,Bayesian Ridge,878581.3193,1444725097516.0923,1181069.2920,-0.1799,0.3845,0.3013,0.0067
lightgbm,Light Gradient Boosting Machine,870969.5246,1364740525846.5576,1145495.2152,-0.1041,0.3734,0.3070,0.1867


In [15]:
# Let’s check the score on the test set:
prediction_holdout = predict_model(best);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Orthogonal Matching Pursuit,739030.5853,893562670954.9481,945284.4392,-0.4902,0.3489,0.3299


In [16]:
# generate predictions on the original dataset
predictions = predict_model(best, data=df_200_155)
# add a date column in the dataset
predictions['Date'] = pd.date_range(start='2016-01-01', end = '2023-06-01', freq = 'MS')
# line plot
fig = px.line(predictions, x='Date', y=["NET_SALES", "Label"], template = 'plotly_dark')
# add a vertical rectange for test-set separation
fig.add_vrect(x0="2022-05-01", x1="2023-07-01", fillcolor="grey", opacity=0.25, line_width=0)
fig.show()

The test era above is depicted by the greyish backdrop near the conclusion - May 2022 and on.

Let’s now finish the model by training the best model, which is Extreme Gradient Boosting, on the complete dataset (this time, including the test set).

In [41]:
final_best = finalize_model(best)

Now that we’ve trained our model on the complete dataset (2016 to Jul 2023), Let’s forecast 5 months into the future through the end of 2023. To utilize our final model to make future predictions, we’ll need to first make a dataset with the Month, Year, and Series columns for future dates.

In [49]:
future_dates = pd.date_range(start = '2023-07-01', end = '2025-12-01', freq = 'MS')
future_df = pd.DataFrame()
future_df['Month'] = [i.month for i in future_dates]
future_df['Year'] = [i.year for i in future_dates]    
future_df['Series'] = np.arange(145, (145+len(future_dates)))
future_df.head()

,Month,Year,Series
0,7,2023,145
1,8,2023,146
2,9,2023,147
3,10,2023,148
4,11,2023,149


In [50]:
predictions_future = predict_model(final_best, data=future_df)
predictions_future.head()

,Month,Year,Series,Label
0,7,2023,145,2229935.00
1,8,2023,146,2619022.75
2,9,2023,147,3053146.50
3,10,2023,148,1879930.50
4,11,2023,149,2904365.75


In [51]:
concat_df = pd.concat([df_200_155,predictions_future], axis=0)
concat_df_i = pd.date_range(start='2016-01-01', end = '2025-12-01', freq = 'MS')
concat_df.set_index(concat_df_i, inplace=True)
fig = px.line(concat_df, x=concat_df.index, y=["NET_SALES", "Label"], template = 'plotly_dark')
fig.show()

In [56]:
predictions_future.to_csv('python200155.csv')

###############
### Phase 2 ###
###############

Forecast for multiple series

In [75]:
# head and tail
print(result_df)
print(result_df.dtypes)

      DEP_ENT      MONTH    NET_SALES  BUDGET_AMOUNT  FORECAST
0     240_155 2017-02-01   4153796.71      4198000.0       NaN
1     240_155 2017-03-01   5940741.71      4892400.0       NaN
2     240_155 2017-04-01   4789757.73      5580200.0       NaN
3     240_155 2017-05-01   6356822.97      5298000.0       NaN
4     240_155 2017-06-01   5756533.29      4678000.0       NaN
...       ...        ...          ...            ...       ...
1260  210_155 2016-07-01   6283148.80     11311652.0       NaN
1261  210_155 2016-08-01   5626725.62     11326527.0       NaN
1262  210_155 2016-09-01  10283086.45     11377763.0       NaN
1263  210_155 2016-10-01   6239512.66     11377763.0       NaN
1264  210_155 2016-11-01  16467966.49     11690688.0       NaN

[1265 rows x 5 columns]
DEP_ENT                  object
MONTH            datetime64[ns]
NET_SALES               float64
BUDGET_AMOUNT           float64
FORECAST                float64
dtype: object


In [76]:
df_all = result_df[result_df['MONTH'] <= pd.Timestamp(2023,6,1)] 
"""
ABOVE: we have data beyond july '23, but we want to compare with 
some models that only had through june. No way to check model accuracy beyond the daya we have
"""
df_all = df_all[["DEP_ENT", "MONTH", "NET_SALES"]]
print(df_all)
print(df_all.dtypes)

      DEP_ENT      MONTH    NET_SALES
0     240_155 2017-02-01   4153796.71
1     240_155 2017-03-01   5940741.71
2     240_155 2017-04-01   4789757.73
3     240_155 2017-05-01   6356822.97
4     240_155 2017-06-01   5756533.29
...       ...        ...          ...
1260  210_155 2016-07-01   6283148.80
1261  210_155 2016-08-01   5626725.62
1262  210_155 2016-09-01  10283086.45
1263  210_155 2016-10-01   6239512.66
1264  210_155 2016-11-01  16467966.49

[1183 rows x 3 columns]
DEP_ENT              object
MONTH        datetime64[ns]
NET_SALES           float64
dtype: object


In [77]:
# ML Can't deal with dates directly so we need to extract some basic properties

# extract month and year from dates
df_all['Month'] = [i.month for i in df_all['MONTH']]
df_all['Year'] = [i.year for i in df_all['MONTH']]
# create a sequence of numbers
#df_all['Series'] = np.arange(1,len(df_all)+1)
print(df_all.head())
print(df_all.dtypes)

   DEP_ENT      MONTH   NET_SALES  Month  Year
0  240_155 2017-02-01  4153796.71      2  2017
1  240_155 2017-03-01  5940741.71      3  2017
2  240_155 2017-04-01  4789757.73      4  2017
3  240_155 2017-05-01  6356822.97      5  2017
4  240_155 2017-06-01  5756533.29      6  2017
DEP_ENT              object
MONTH        datetime64[ns]
NET_SALES           float64
Month                 int64
Year                  int64
dtype: object


In [78]:
# check the unique time_series - 14 different department-entity combinations
df_all['DEP_ENT'].nunique()

14

In [79]:
# plot multiple time series with moving avgs in a loop - just a test

for i in df_all['DEP_ENT'].unique():
    subset = df_all[df_all['DEP_ENT'] == i]
    subset['moving_average'] = subset['NET_SALES'].rolling(30).mean()
    fig = px.line(subset, x="MONTH", y=["NET_SALES","moving_average"], title = i, template = 'plotly_dark')
    
fig.show()

In [67]:
from tqdm import tqdm

In [83]:
all_ts = df_all['DEP_ENT'].unique()

all_results = []
final_model = {}

for i in tqdm(all_ts):
    
    df_subset = df_all[df_all['DEP_ENT'] == i]
    
    # initialize setup from pycaret.regression
    s = setup(df_subset, target = 'NET_SALES', train_size = 0.8,
              data_split_shuffle = False, fold_strategy = 'timeseries', fold = 3,
              ignore_features = ['MONTH', 'DEP_ENT'],
              numeric_features = ['Year'],
              categorical_features = ['Month'],
              silent = True, verbose = False, session_id = 123)
    
    # compare all models and select best one based on MAE
    best_model = compare_models(sort = 'MAE', verbose=False)
    
    # capture the compare result grid and store best model in list
    p = pull().iloc[0:1]
    p['DEP_ENT'] = str(i)
    all_results.append(p)
    
    # finalize model i.e. fit on entire data including test set
    f = finalize_model(best_model)
    
    # attach final model to a dictionary
    final_model[i] = f
    
    # save transformation pipeline and model as pickle file 
    save_model(f, model_name='trained_models/' + str(i), verbose=False)

  0%|          | 0/14 [00:00<?, ?it/s]


AttributeError: 'Simple_Imputer' object has no attribute 'fill_value_categorical'